In [5]:

# GBDT
from sklearn.ensemble import GradientBoostingRegressor
# XGBoost
import xgboost as xgb
# LightGBM
import lightgbm as lgb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle
import multiprocessing
from sklearn.preprocessing import StandardScaler
ss = StandardScaler() 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,LinearRegression,LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest

In [1]:
import pandas as pd
## 先导入之前处理好的数据
train_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/train_data.csv'
test_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/test_a.csv'
data_train_origin = pd.read_csv(train_path)
data_train = data_train_origin.copy()
data_train['Type'] = 'Train'
data_test_origin = pd.read_csv(test_path)
data_test = data_test_origin.copy()
data_test['Type'] = 'Test'
data_all = pd.concat([data_train, data_test], ignore_index=True)

/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [2]:
data_all.head()

,ID,Type,area,bankNum,buildYear,busStationNum,city,communityName,drugStoreNum,gymNum,...,totalWorkers,tradeLandArea,tradeLandNum,tradeMeanPrice,tradeMoney,tradeNewMeanPrice,tradeNewNum,tradeSecNum,tradeTime,uv
0,100309852,Train,68.06,16,1953,36,SH,XQ00051,12,15,...,28248,0.0,0,47974.22551,2000.0,104573.48460,25,111,2018/11/28,284.0
1,100307942,Train,125.55,16,2007,184,SH,XQ00130,27,5,...,14823,0.0,0,40706.66775,2000.0,33807.53497,2,2,2018/12/16,22.0
2,100307764,Train,132.00,37,暂无信息,60,SH,XQ00179,24,35,...,77645,0.0,0,34384.35089,16000.0,109734.16040,11,555,2018/12/22,20.0
3,100306518,Train,57.00,47,暂无信息,364,SH,XQ00313,83,30,...,8750,108037.8,1,20529.55050,1600.0,30587.07058,58,260,2018/12/21,279.0
4,100305262,Train,129.00,10,暂无信息,141,SH,XQ01257,21,5,...,800,0.0,0,24386.36577,2900.0,51127.32846,34,38,2018/11/18,480.0


In [3]:



#对数据进行处理
def preprocessingData(data):
    # 填充缺失值
    data['rentType'][data['rentType'] == '--'] = '未知方式'  #进行填充
    
    # 转换object类型数据
    columns = ['rentType','communityName','houseFloor', 'houseToward', 'houseDecoration',  'region', 'plate']
    
    for feature in columns:
        data[feature] = LabelEncoder().fit_transform(data[feature])

    # 将buildYear列转换为整型数据
    buildYearmean = pd.DataFrame(data[data['buildYear'] != '暂无信息']['buildYear'].mode())  #用平均数填充
    data.loc[data[data['buildYear'] == '暂无信息'].index, 'buildYear'] = buildYearmean.iloc[0, 0]
    data['buildYear'] = data['buildYear'].astype('int')  #年份转成整数

    # 处理pv和uv的空值
    data['pv'].fillna(data['pv'].mean(), inplace=True)
    data['uv'].fillna(data['uv'].mean(), inplace=True)
    data['pv'] = data['pv'].astype('int')
    data['uv'] = data['uv'].astype('int')

    # 分割交易时间
    def month(x):
        month = int(x.split('/')[1])
        return month
    def year(x):
        year = int(x.split('/')[0])
        return year
    data['month'] = data['tradeTime'].apply(lambda x: month(x))
    data['year'] = data['tradeTime'].apply(lambda x: year(x))
    
    # 去掉部分特征
    data.drop('city', axis=1, inplace=True)
    data.drop('tradeTime', axis=1, inplace=True)
    data.drop('ID', axis=1, inplace=True)
    return data



In [6]:
data_train = preprocessingData(data_train)
data_test = preprocessingData(data_test)

/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
def IF_drop(train):
    IForest = IsolationForest(contamination=0.01)
    IForest.fit(train["tradeMoney"].values.reshape(-1,1))
    y_pred = IForest.predict(train["tradeMoney"].values.reshape(-1,1))
    drop_index = train.loc[y_pred==-1].index
    print(drop_index)
    train.drop(drop_index,inplace=True)
    return train

In [9]:
#data_train = IF_drop(data_train)
#data_test = IF_drop(data_test)

KeyError: 'tradeMoney'

In [11]:
def dropData(train):  
    # 丢弃部分异常值
    train = train[train.area <= 200]
    train = train[(train.tradeMoney <=16000) & (train.tradeMoney >=700)]
    train.drop(train[(train['totalFloor'] == 0)].index, inplace=True)
    return train  
#数据集异常值处理
data_train = dropData(data_train)
#data_test = dropData(data_test)

In [13]:
def newfeature(data):


    # 将houseType转为'Room'，'Hall'，'Bath'    用于判断单个房间数量对结果的影响
    
    def Room(x):
        Room = int(x.split('室')[0])
        return Room
    def Hall(x):
        Hall = int(x.split("室")[1].split("厅")[0])
        return Hall
    def Bath(x):
        Bath = int(x.split("室")[1].split("厅")[1].split("卫")[0])
        return Bath

    data['Room'] = data['houseType'].apply(lambda x: Room(x))
    data['Hall'] = data['houseType'].apply(lambda x: Hall(x))
    data['Bath'] = data['houseType'].apply(lambda x: Bath(x))
    data['Room_Bath'] = (data['Bath']+1) / (data['Room']+1)
    # 填充租房类型    根据
    



    # data['day'] = data['tradeTime'].apply(lambda x: day(x))# 结果变差
    #     data['pv/uv'] = data['pv'] / data['uv']
    #     data['房间总数'] = data['室'] + data['厅'] + data['卫']

    # 合并部分配套设施特征
    data['trainsportNum'] = 5 * data['subwayStationNum'] / data['subwayStationNum'].mean() + data['busStationNum'] / \
                                                                                             data[
                                                                                                 'busStationNum'].mean()
    data['all_SchoolNum'] = 2 * data['interSchoolNum'] / data['interSchoolNum'].mean() + data['schoolNum'] / data[
        'schoolNum'].mean() \
                            + data['privateSchoolNum'] / data['privateSchoolNum'].mean()
    data['all_hospitalNum'] = 2 * data['hospitalNum'] / data['hospitalNum'].mean() + \
                              data['drugStoreNum'] / data['drugStoreNum'].mean()
    data['all_mall'] = data['mallNum'] / data['mallNum'].mean() + \
                       data['superMarketNum'] / data['superMarketNum'].mean()
    data['otherNum'] = data['gymNum'] / data['gymNum'].mean() + data['bankNum'] / data['bankNum'].mean() + \
                       data['shopNum'] / data['shopNum'].mean() + 2 * data['parkNum'] / data['parkNum'].mean()

    data.drop(['subwayStationNum', 'busStationNum','houseType',
               'interSchoolNum', 'schoolNum', 'privateSchoolNum',
               'hospitalNum', 'drugStoreNum', 'mallNum', 'superMarketNum', 'gymNum', 'bankNum', 'shopNum', 'parkNum'],
              axis=1, inplace=True)
    # 提升0.0005
    
#     data['houseType_1sumcsu']=data['Bath'].map(lambda x:str(x))+data['month'].map(lambda x:str(x))
#     data['houseType_2sumcsu']=data['Bath'].map(lambda x:str(x))+data['communityName']
#     data['houseType_3sumcsu']=data['Bath'].map(lambda x:str(x))+data['plate']
    
    #data.drop('houseType', axis=1, inplace=True)
    #data.drop('tradeTime', axis=1, inplace=True)
    
    data["area"] = data["area"].astype(int)


    # categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName','region', 'plate']
    categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration',  'region', 'plate','cluster']

    return data
data_train = newfeature(data_train)
data_test = newfeature(data_test)

In [15]:
data_train['tradeMoney'].head()

0     2000.0
1     2000.0
2    16000.0
3     1600.0
4     2900.0
Name: tradeMoney, dtype: float64

In [16]:
data_test.head()

,area,rentType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,region,plate,buildYear,...,year,Room,Hall,Bath,Room_Bath,trainsportNum,all_SchoolNum,all_hospitalNum,all_mall,otherNum
0,36,2,2,21,6,0,2236,11,11,2008,...,2018,1,1,1,1.000000,3.788256,5.545572,4.332163,0.892137,4.635644
1,64,2,2,11,3,0,494,1,52,2009,...,2018,2,1,1,0.666667,10.132852,1.276564,4.055809,1.844215,4.801505
2,98,2,2,6,4,3,512,1,52,1996,...,2018,3,1,1,0.500000,10.132852,1.276564,4.055809,1.844215,4.801505
3,43,2,2,6,4,0,476,1,52,1994,...,2018,1,1,1,1.000000,10.132852,1.276564,4.055809,1.844215,4.801505
4,92,2,2,6,4,0,770,1,58,2004,...,2018,2,2,1,0.666667,5.377752,3.409058,1.371388,2.436830,4.902879


In [17]:
train_clean_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/train_data_clean.csv'
test_clean_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/test_a_clean.csv'

In [18]:
data_train.to_csv(train_clean_path)
data_test.to_csv(test_clean_path)